In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
data = pd.read_csv('./data/ma405_tick.csv')

data.tail()

,date,volume,turnover,open_interest,last_price,bid_price,ask_price,bid_volume,ask_volume
811386,2024-05-17 14:59:52,0,0.0,1168.0,2755.0,2684.0,2755.0,44,20
811387,2024-05-17 14:59:56,0,0.0,1168.0,2755.0,2685.0,2755.0,11,20
811388,2024-05-17 14:59:57,0,0.0,1168.0,2755.0,2685.0,2755.0,44,20
811389,2024-05-17 14:59:58,0,0.0,1168.0,2755.0,2684.0,2755.0,44,20
811390,2024-05-17 15:00:00,0,0.0,1168.0,2755.0,2684.0,2755.0,44,20


In [3]:
def argmax(x):
    return x.argmax()
def argmin(x):
    return x.argmin()
def get_diff(x):
    if abs(x[0])>abs(x[1]):
        return x[0]
    else:
        return x[1]
data['volume_delta'] = data['volume'].diff()

In [116]:
p0 = 60
p1 = 360

roll_0 = data['last_price'].rolling(p0)
roll_1 = data['last_price'].rolling(p1)

max0 = roll_0.max()
min0 = roll_0.min()

max1 = roll_1.max()
min1 = roll_1.min()

data['min0_diff'] = data['last_price'] - max0
data['max0_diff'] = data['last_price'] - min0

data['min1_diff'] = data['last_price'] - max1 
data['max1_diff'] = data['last_price'] - min1

data['diff0']= data[['min0_diff','max0_diff']].apply(get_diff,axis=1)
data['diff1']= data[['min1_diff','max1_diff']].apply(get_diff,axis=1)


In [117]:
# long_benifit=6 , long_loss = -8 short_benifit = 3, short_loss = -4
# long盈利71，交易次数91， short盈利14，交易次数33
h1 = data[(data['last_price']<=data['bid_price'])&(data['diff0']>0)&(data['bid_volume']>3*data['ask_volume'])].index
l1 = data[(data['last_price']>=data['ask_price'])&(data['diff0']<0)&(data['ask_volume']>3.8*data['bid_volume'])].index


l1_close = data[(data['bid_volume']>4*data['ask_volume'])&(data['diff1']>=3)].index
h1_close = data[(data['ask_volume']>4*data['bid_volume'])&(data['diff1']<=-3)].index


In [118]:
data['sign']=0
data['close_sign'] = 0
data['sign'][h1] = 1
data['sign'][l1] = -1

data['close_sign'][h1_close] = -1
data['close_sign'][l1_close] = 1

test_data = data[-880000:-20000]

C:\Users\rcl\AppData\Local\Temp\ipykernel_15400\2524769255.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sign'][h1] = 1
C:\Users\rcl\AppData\Local\Temp\ipykernel_15400\2524769255.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sign'][l1] = -1
C:\Users\rcl\AppData\Local\Temp\ipykernel_15400\2524769255.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['close_sign'][h1_close] = -1
C:\Users\rcl\AppData\Local\Te

In [119]:
pos = 0

long_benifit = 15
long_loss = -19
short_benifit = 15
short_loss = -19
long_diff = []
short_diff = []
total_diff = []
direction = []
pos_price = 0

for row in tqdm(test_data.iterrows()):
    sign = row[1]['sign']
    
    if len(total_diff)>=2:
        if total_diff[-1]<0 and total_diff[-2]<0:
            if direction[-1] == direction[-2] == sign:
                sign*=-1
         
    if pos == 0:
        if sign == 1:
            pos = 1
            pos_price = row[1]['ask_price']
            direction.append(1)
        elif sign == -1:
            pos = -1
            pos_price = row[1]['bid_price']
            direction.append(-1)
    elif pos == 1 and row[1]['bid_price']>0:
        diff = row[1]['bid_price'] - pos_price
        if diff>=long_benifit and row[1]['close_sign'] == -1:
            long_diff.append(diff)
            pos = 0
            total_diff.append(diff)
        elif diff<= long_loss:
            long_diff.append(diff)
            pos = 0
            total_diff.append(diff)
    elif pos == -1 and row[1]['ask_price']>0:
        diff = pos_price - row[1]['ask_price']
        if diff>=short_benifit and row[1]['close_sign'] == 1:
            short_diff.append(diff)
            pos = 0
            total_diff.append(diff)
        elif diff<= short_loss:
            short_diff.append(diff)
            pos = 0
            total_diff.append(diff)
        
print(sum(long_diff),len(long_diff))
print(sum(short_diff),len(short_diff))

791391it [00:29, 26635.93it/s]

112.0 48
110.0 25


In [111]:
for i,j in zip(direction,total_diff):
    print(i,j)

1 20.0
-1 25.0
1 -20.0
-1 -19.0
-1 17.0
-1 -19.0
1 -19.0
-1 17.0
1 15.0
1 -19.0
-1 -19.0
1 19.0
1 -19.0
1 15.0
1 -20.0
-1 -19.0
1 23.0
-1 15.0
1 -19.0
-1 16.0
1 -19.0
-1 -19.0
1 16.0
1 19.0
1 15.0
-1 20.0
1 25.0
1 21.0
1 19.0
1 -20.0
-1 21.0
1 -21.0
1 -19.0
-1 17.0
1 -19.0
-1 17.0
1 -19.0
-1 -19.0
1 17.0
1 15.0
1 18.0
1 -19.0
1 -19.0
-1 -19.0
1 -19.0
-1 19.0
1 -19.0
1 16.0
1 18.0
1 -19.0
1 17.0
1 15.0
1 18.0
1 15.0
-1 16.0
1 22.0
1 15.0
1 26.0
1 29.0
-1 15.0
-1 18.0
1 18.0
-1 -19.0
1 -19.0
-1 18.0
1 -19.0
-1 15.0
1 -19.0
-1 15.0
1 -19.0
1 33.0
-1 -19.0
1 17.0
